<h1>makemore part2</h1>
/content/drive/MyDrive/Colab Notebooks/makemore_part3_part2.ipynb

<p>1:05:08 REsnet review of layers weight, linear, batch, nonlinear. Deep NN example of different layers. Conv layers depth and bias=False bc after batch normalization adds bias and removes the need for additional bias. </p>

<p>1:13:12 a momentum of 0.1 seems too big. If big the batch wont converge to the true mean and std of the training data. Will thrash around the true mean/std. </p>

<p></p>

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

ValueError: mount failed

In [5]:
%cd /content/drive/MyDrive/makemore

In [ ]:
#pytorch resnet

In [8]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt


class Linear:
    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn( (fan_in, fan_out), generator = g) / fan_in ** 0.5
        self.bias = torch.randn(fan_out)

    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out

    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])

class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum = 0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)

    def __call__(self, x):
      # forward
      if self.training:
        xmean = x.mean(0, keepdim=True) # batch mean
        xvar = x.var(0, keepdim=True, unbiased=True)
      else:
        xmean = self.running_mean
        xvar = self.running_var

      xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
      self.out = self.gamma * xhat + self.beta

      if self.training:
        with torch.no_grad():
          self.running_mean =  (1 - self.momentumn) * self.running_mean + self.momentum * xmean
          self.running_var = (1 - self.momentumn) * self.running_var + self.momentum * xvar
      return self.out

class Tanh:
    def __call__(self,x):
        self.out = torch.tanh(x)
        return self.out
    def parameters(self):
        return []


# read in all the words
words = open('names.txt', 'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

print(f"vocab_size:{vocab_size}")

block_size = 3
n_embed = 10
n_hidden = 100
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed), generator = g)

layers = [
  Linear( n_embed*block_size, n_hidden), Tanh(),
  Linear( n_hidden, n_hidden), Tanh(),
  Linear( n_hidden, n_hidden), Tanh(),
  Linear( n_hidden, n_hidden), Tanh(),
  Linear( n_hidden, n_hidden), Tanh(),
  Linear( n_hidden, vocab_size),
]


with torch.no_grad():
    # last layer make less confident??????
    layers[-1].weight *= 0.1
    # all other layers apply gain
    for layer in layers[:-1]:
        if isinstance(layer, Linear):
            layer.weight *= 5/3


parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum( p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True





vocab_size:27
46497
